In [1]:
import numpy as np

In [2]:
class ANN:
    #List of List for each layer along with their information
    #Shape: [ [layerType,activationF(),neuronCount],[layerType,activationF(),neuronCount]]
    layer_info = None
    #List of weight matrices of each layer
    #Shape: dxk (where d is no of input)(k is no of output)
    w_l_ij= None
    #List of bias vector of each layer
    #Shape: 1xk (where k is no of output/neuron)
    b_l_j = None
    #List of output of each neuron
    #Shape: 1xd (where d is dimension)
    a_l_ij = None
    #List of d(a_l+1)/da_l (where a_l+1 is output of layer)(a_l is input of layer)
    #Shape: 1xd (same shape as input of layer)
    delta_l_ij = None
    #List of matrices of dl/dw for each layer
    dl_db = None
    dl_dw = None
    #Dictionary for differentiation function for a input function. diff[f()] => df()
    diff = {}
    input_featureCount = None
    input_count = None
    def __init__(self,xShape,rseed = 10,listLayer= None):
        np.random.seed(rseed)
        inputCount,inputDim = xShape
        self.input_featureCount = inputDim
        self.input_count = inputCount
        self.layer_info = [['None','None',self.input_featureCount]]
        self.w_l_ij = ['None']
        self.b_l_j = ['None']
        self.optimizer = None
        self.diff[ANN.activation_linear] = ANN.diff_activation_linear
        self.diff[ANN.activation_sigmoid] = ANN.diff_activation_sigmoid
        self.diff[ANN.activation_tanh] = ANN.diff_activation_tanh

    def compile(self,optimizer,lossFunction):
        self.optimizer = optimizer
        self.layer_info.append(['Loss',lossFunction])

    def __str__(self):
        return

    def addLayers_Dense(self,neuronCount,activationFunction):
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.layer_info[-2][2],neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)

    def addLayers_softmax(self):
        self.layer_info.append(['Softmax',self.layer_info[-1][2]])

    def forward_layer_dense(self,layerNo):
        if layerNo < 1:
            print("ERRRROR")
        
        #Output = activationFunction(a_l-1 @ w_l + b_l)
        output = self.layer_info[layerNo][1](self.a_l_ij[layerNo-1] @ self.w_l_ij[layerNo] + self.b_l_j[layerNo]) 
        self.a_l_ij.append(output)
    
    def backward_layer_dense(self,layerNo):
        #at each layer we need to calculate da_l/da_l-1, and da_l/dw_l

        #Shape: dxk
        da_1da = self.w_l_ij[layerNo].T
        #Shape: dxk
        da_l1_dw_l = np.copy(self.a_l_ij[layerNo-1]).T
        for i in range(self.w_l_ij[layerNo].shape[1]-1):
            da_l1_dw_l = np.hstack((da_l1_dw_l,self.a_l_ij[layerNo-1].T))

        #multiply both by dL/da_l to get dL/da_l-1 and dL/dw_l
        self.delta_l_ij[layerNo] = self.delta_l_ij[layerNo+1] @ da_1da
        #print(da_l1_dw_l,"\n", self.delta_l_ij[layerNo+1])
        temp = np.zeros(self.w_l_ij[layerNo].shape)
        for i in range(self.w_l_ij[layerNo].shape[0]):
            for j in range(self.w_l_ij[layerNo].shape[1]):
                temp[i][j] = da_l1_dw_l[i][j] * self.delta_l_ij[layerNo][0][j]

        self.dl_dw.append(temp)
        self.dl_db.append(self.delta_l_ij[layerNo+1])
    
    def forward_layer_softmax(self,layerNo):
        if layerNo < 1: 
            print("ERRORRR")
        denom = np.sum( np.exp(self.a_l_ij[layerNo-1]) )
        output = np.exp(self.a_l_ij[layerNo]) / denom
        self.a_l_ij.append(output)

    def backward_layer_softmax(self,layerNo):
        pass

    #Create cross entropy loss 
    def forward_layer_loss(self,layerNo,y):
        y_hat  =  self.a_l_ij[-1]
        lossFunction = self.layer_info[layerNo][1]
        return lossFunction(y_hat,y)

    def forwardProp(self,x,y):
        self.a_l_ij = []
        self.a_l_ij.append(x)
        for layerNo in range(len(self.layer_info)):
            if self.layer_info[layerNo][0] == 'Dense':
                self.forward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Softmax':
                self.forward_layer_softmax(layerNo)
            if self.layer_info[layerNo][0] == 'Loss':
                cost = self.forward_layer_loss(layerNo,y)
                return cost
            
    #Handle softmax backpropogation
    def backwardProp(self,y):
        self.dl_dw = []
        self.dl_db = []
        lastIndex = len(self.layer_info) - 2
        self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        
        #MSE LOSS And Dense
        self.delta_l_ij[lastIndex] = 2 * ( (self.a_l_ij[lastIndex] - y) @ self.w_l_ij[lastIndex].T) 
        self.dl_dw.append(2*(self.a_l_ij[lastIndex-1].T @ self.a_l_ij[lastIndex] - y))
        self.dl_db.append(2*(self.a_l_ij[lastIndex] - y))

        #BackPropogation
        for layerNo in range(lastIndex-1,0,-1):
            if self.layer_info[layerNo][0] == 'Dense':
                self.backward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Softmax':
                self.backward_layer_softmax(layerNo)
                
        self.dl_dw.append(['None'])
        self.dl_db.append(['None'])
        self.dl_dw.reverse()
        self.dl_db.reverse()


            
    def fit(self,x,y):
        #Hyperparameters if required: 
        epoch = 100
        eta = 1e-6
        self.optimizer(self,x,y,eta,epoch)

    @staticmethod    
    def optimizer_gradientDescent(Obj,x,y,eta,epoch):
        for i in  range(epoch):
            errorSum = 0
            print("Epoch:",i,end="\t[")
            for j in range(1):
                #x and y both are 2d matrix
                x_1xd = x[j:j+1]
                y_1xk = y[j:j+1]
                errorSum += Obj.forwardProp(x_1xd,y_1xk)
                Obj.backwardProp(y_1xk)
                for index in range(1,len(Obj.w_l_ij)):
                    Obj.w_l_ij[index] = Obj.w_l_ij[index] - (eta * Obj.dl_dw[index])
                    Obj.b_l_j[index] = Obj.b_l_j[index] - (eta * Obj.dl_db[index] )
            print("]", end='\t')
            print("Error:",errorSum)

    @staticmethod
    def loss_MSE(P,y):
        return np.sum((P - y)**2)   
      
    @staticmethod
    def activation_linear(z):
        return z
    
    @staticmethod
    def diff_activation_linear(x):
        return np.ones(x.shape)
    
    @staticmethod
    def activation_sigmoid(z):    
        return 1 / (1 + np.exp(-z))
    
    @staticmethod    
    def diff_activation_sigmoid(a):
        return a * (1- a)

    @staticmethod
    def activation_tanh(z):
        sinh = np.exp(z) - np.exp(-z)
        cosh = np.exp(z) + np.exp(-z)
        return sinh / cosh
    
    @staticmethod
    def diff_activation_tanh(z):
        return 1 - (z * z)

    @staticmethod
    def activation_relu(z):
        pass

In [3]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [4]:
dictt = sk.fetch_california_housing()
x = dictt.data
y = np.array(dictt.target)
y = y.T
x

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [5]:
# from sklearn.preprocessing import Normalizer
# transformer = Normalizer().fit(x)
# x = transformer.transform(x)
# x

In [6]:
model = ANN(x.shape,20)

In [7]:
model.addLayers_Dense(4,ANN.activation_linear)
model.addLayers_Dense(3,ANN.activation_linear)
model.addLayers_Dense(1,ANN.activation_linear)

In [8]:
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)

In [9]:
model.layer_info

[['None', 'None', 8],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 4],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 3],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 1],
 ['Loss', <function __main__.ANN.loss_MSE(P, y)>]]

In [10]:
import copy
w_old = copy.deepcopy(model.w_l_ij)
b_old = copy.deepcopy(model.b_l_j)
dldw = copy.deepcopy(model.dl_dw)

In [11]:
model.fit(x,y)

Epoch: 0	[]	Error: 128384.96048739352
Epoch: 1	[]	Error: 24495.659407641015
Epoch: 2	[]	Error: 720.1462495051715
Epoch: 3	[]	Error: 79.05511746785547
Epoch: 4	[]	Error: 9.109561952916417
Epoch: 5	[]	Error: 1.0740339161167474
Epoch: 6	[]	Error: 0.13237708407991283
Epoch: 7	[]	Error: 0.018317109942635596
Epoch: 8	[]	Error: 0.0033282021430480263
Epoch: 9	[]	Error: 0.0009799253916001182
Epoch: 10	[]	Error: 0.0004989355779448052
Epoch: 11	[]	Error: 0.00037203543614244437
Epoch: 12	[]	Error: 0.0003330526339760441
Epoch: 13	[]	Error: 0.00032018863137534285
Epoch: 14	[]	Error: 0.00031575940692064744
Epoch: 15	[]	Error: 0.0003141427186268393
Epoch: 16	[]	Error: 0.0003134745899757143
Epoch: 17	[]	Error: 0.0003131277488266941
Epoch: 18	[]	Error: 0.00031289004615144824
Epoch: 19	[]	Error: 0.0003126896214397846
Epoch: 20	[]	Error: 0.0003125021251008625
Epoch: 21	[]	Error: 0.00031231930659095166
Epoch: 22	[]	Error: 0.0003121383705452513
Epoch: 23	[]	Error: 0.0003119583697446308
Epoch: 24	[]	Error: 0

In [12]:
model.a_l_ij

[array([[   8.3252    ,   41.        ,    6.98412698,    1.02380952,
          322.        ,    2.55555556,   37.88      , -122.23      ]]),
 array([[-121.13504572,  154.22447139,  -33.54439386,  -42.03656395]]),
 array([[-31.32529087,   4.57665221,  16.27493424]]),
 array([[4.50869225]])]

In [13]:
w_old

['None',
 array([[0.5881308 , 0.89771373, 0.89153073, 0.81583748],
        [0.03588959, 0.69175758, 0.37868094, 0.51851095],
        [0.65795147, 0.19385022, 0.2723164 , 0.71860593],
        [0.78300361, 0.85032764, 0.77524489, 0.03666431],
        [0.11669374, 0.7512807 , 0.23921822, 0.25480601],
        [0.85762553, 0.94977903, 0.56168686, 0.17878052],
        [0.77025193, 0.49238104, 0.63125307, 0.83949792],
        [0.4610394 , 0.49794007, 0.67941112, 0.65078591]]),
 array([[0.32920641, 0.51064106, 0.26362883],
        [0.31051155, 0.62685344, 0.55744981],
        [0.31857956, 0.39484322, 0.25797459],
        [0.58224112, 0.16162871, 0.59813382]]),
 array([[0.40864343],
        [0.7786879 ],
        [0.80397057]])]

In [14]:
model.b_l_j

['None',
 array([[0.26848432, 0.06681538, 0.77116834, 0.48061691]]),
 array([[0.82564995, 0.15608496, 0.73398557]]),
 array([[0.78575175]])]

In [15]:
b_old

['None',
 array([[0.26879524, 0.06732467, 0.77144514, 0.48098413]]),
 array([[0.82582358, 0.15639172, 0.73430052]]),
 array([[0.78607144]])]

In [16]:
model.dl_dw

[['None'],
 array([[ -0.48378502,  -0.24081496,  -0.27184988,  -0.36823632],
        [ -2.38254764,  -1.18596712,  -1.33880811,  -1.81349265],
        [ -0.40585403,  -0.20202305,  -0.22805868,  -0.30891861],
        [ -0.05949451,  -0.02961474,  -0.03343133,  -0.04528466],
        [-18.71171563,  -9.3141808 , -10.51454171, -14.24255203],
        [ -0.14850568,  -0.07392207,  -0.08344874,  -0.11303613],
        [ -2.20124158,  -1.09571792,  -1.23692807,  -1.67549028],
        [  7.10289752,   3.53562832,   3.99128085,   5.40641967]]),
 array([[ 2.03547526,  2.69771016,  1.79750515],
        [-2.59148865, -3.4346206 , -2.28851427],
        [ 0.56365838,  0.7470427 ,  0.49776033],
        [ 0.70635534,  0.93616562,  0.6237744 ]]),
 array([[-291.52419252],
        [  32.21743272],
        [ 137.70533983]])]

In [17]:
dldw

In [18]:
# self.delta_l_ij = [i for i in range(0,lastIndex+1)]
# self.delta_l_ij[lastIndex] = (2 * (self.a_l_ij[lastIndex] - y).T * self.diff[self.layer_info[lastIndex][1]](self.a_l_ij[lastIndex]))
# self.dl_dw.append(self.a_l_ij[lastIndex-1].T @ self.delta_l_ij[lastIndex])
# for layerNo in range(lastIndex-1,0,-1):
#     self.delta_l_ij[layerNo] = (self.w_l_ij[layerNo+1] @self.delta_l_ij[layerNo+1].T).T * self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo])
#     self.dl_dw.append(self.a_l_ij[layerNo-1].T @ self.delta_l_ij[layerNo])
# self.dl_dw.append(['None'])
# self.dl_dw.reverse()

In [19]:
        # self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        # self.delta_l_ij[lastIndex] = (2 * (self.a_l_ij[lastIndex] - y).T * self.diff[self.layer_info[lastIndex][1]](self.a_l_ij[lastIndex]))
        # self.dl_dw.append(self.a_l_ij[lastIndex-1].T @ self.delta_l_ij[lastIndex])
        # self.dl_db.append(self.a_l_ij[lastIndex])
        # for layerNo in range(lastIndex-1,0,-1):
        #     self.delta_l_ij[layerNo] = (self.w_l_ij[layerNo+1] @self.delta_l_ij[layerNo+1].T).T * self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo])
        #     self.dl_dw.append(self.a_l_ij[layerNo-1].T @ self.delta_l_ij[layerNo])
        #     self.dl_db.append(self.delta_l_ij[layerNo])
        # self.dl_dw.append(['None'])
        # self.dl_db.append(['None'])
        # self.dl_dw.reverse()
        # self.dl_dw.reverse()
        

In [20]:
x = np.array([[1,2,3]])
x_ = np.hstack((x.T,x.T))
x_

array([[1, 1],
       [2, 2],
       [3, 3]])

Activation Function (Backpropogation)
MSE -> Generalise for any loss
Softmax backProp
Stopping condition stochastic gd
